In [874]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [875]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test_temp=test.copy()
train_temp=train.copy()

In [876]:
train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [877]:
test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [878]:
(len(train[train.is_promoted == 1])/len(train))*100

8.517004816815064

In [879]:
train['education']=train['education'].fillna("Bachelor's")
train['previous_year_rating']=train['previous_year_rating'].fillna(3)

In [880]:
test['education']=test['education'].fillna("Bachelor's")
test['previous_year_rating']=test['previous_year_rating'].fillna(3)

In [881]:
#Following features are created based on domain importance combinations
train['total_score']=train['no_of_trainings']*train['avg_training_score']
test['total_score']=test['no_of_trainings']*test['avg_training_score']
train['feat1']=train['KPIs_met >80%']+train['awards_won?']
test['feat1']=test['KPIs_met >80%']+test['awards_won?']

In [882]:
train1=pd.get_dummies(train,columns=['department','recruitment_channel','region', 'education', 'gender'],prefix_sep='_',drop_first=True)

In [883]:
test1=pd.get_dummies(test,columns=['department','recruitment_channel', 'region', 'education', 'gender'],prefix_sep='_',drop_first=True)

In [884]:
target=train['is_promoted']

In [885]:
f_train=train1.drop('is_promoted',axis=1)

In [886]:
f_train=f_train.drop(['employee_id'],axis=1)

In [887]:
test1=test1.drop(['employee_id'],axis=1)

In [888]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

In [889]:
X_train,X_test,Y_train,Y_test=train_test_split(f_train,target,test_size=.4)

In [890]:
#Hyperparameters are tuned by Gridsearch in another notebook.Precise value are selected based on accuracy.
xgb = XGBClassifier(
 learning_rate =0.1,
 n_estimators=700,
 max_depth=3,
 min_child_weight=1,
 gamma=1,
 subsample=0.9,
 colsample_bytree=0.9,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=2.1,
 seed=27)

In [891]:
xgb.fit(np.array(f_train),target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=700,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=2.1, seed=27,
       silent=True, subsample=0.9)

In [892]:
test_temp['is_promoted']=xgb.predict(np.array(test1))

C:\Users\sanjaydeo96\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [893]:
test_temp=test_temp[['employee_id','is_promoted']]

In [894]:
test_temp.to_csv('subm_final1.csv',index=False)